In [1]:
# !pip install yfinance

In [7]:
import yfinance as yf
from pyspark.sql import SparkSession


# Create a Spark session
spark = SparkSession.builder.appName("StocksDataFrameToCSV").getOrCreate()

24/04/14 02:34:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [8]:
df = yf.download("AMZN")

[*********************100%%**********************]  1 of 1 completed


In [9]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6772 entries, 1997-05-15 to 2024-04-12
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       6772 non-null   float64
 1   High       6772 non-null   float64
 2   Low        6772 non-null   float64
 3   Close      6772 non-null   float64
 4   Adj Close  6772 non-null   float64
 5   Volume     6772 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 370.3 KB


In [11]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,6772.000000,6772.000000,6772.000000,6772.000000,6772.000000,6.772000e+03
mean,35.736946,36.162943,35.279714,35.731408,35.731408,1.391249e+08
std,51.740289,52.338296,51.096551,51.722972,51.722972,1.386093e+08
min,0.070313,0.072396,0.065625,0.069792,0.069792,9.744000e+06
25%,2.060985,2.100000,2.019875,2.059500,2.059500,6.602150e+07
50%,8.118750,8.249750,8.038500,8.154000,8.154000,1.029480e+08
75%,48.899627,49.152125,48.387501,48.827376,48.827376,1.575920e+08
max,187.720001,189.770004,185.509995,189.050003,189.050003,2.086584e+09


In [7]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1997-05-15,0.121875,0.125000,0.096354,0.097917,0.097917,1443120000
1997-05-16,0.098438,0.098958,0.085417,0.086458,0.086458,294000000
1997-05-19,0.088021,0.088542,0.081250,0.085417,0.085417,122136000
1997-05-20,0.086458,0.087500,0.081771,0.081771,0.081771,109344000
1997-05-21,0.081771,0.082292,0.068750,0.071354,0.071354,377064000


In [12]:
# Convert to a Pyspark Dataframe
spark_df = spark.createDataFrame(df)

In [14]:
# Check the schema
spark_df.printSchema()
spark_df.show()

root
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: long (nullable = true)



+-------------------+-------------------+-------------------+-------------------+-------------------+----------+
|               Open|               High|                Low|              Close|          Adj Close|    Volume|
+-------------------+-------------------+-------------------+-------------------+-------------------+----------+
|0.12187500298023224|              0.125|0.09635400027036667|0.09791699796915054|0.09791699796915054|1443120000|
|0.09843800216913223|0.09895800054073334| 0.0854170024394989| 0.0864579975605011| 0.0864579975605011| 294000000|
|0.08802100270986557|0.08854199945926666|0.08124999701976776| 0.0854170024394989| 0.0854170024394989| 122136000|
| 0.0864579975605011|0.08749999850988388|0.08177100121974945|0.08177100121974945|0.08177100121974945| 109344000|
|0.08177100121974945|0.08229199796915054|0.06875000149011612|0.07135400176048279|0.07135400176048279| 377064000|
|0.07187499850988388|0.07239600270986557|0.06562499701976776| 0.0697920024394989| 0.069792002439

In [16]:
# generate a CSV and save it to HDFS
spark_df.write.csv('hdfs:///ca3/amazon_2.csv')